# Suramerica Results v2.2 Completado

Cargamos los resultados de iterar todos los conjuntos de indicadores

In [1]:
import pandas as pd
import numpy as np
import time
import plotly.plotly as py
import plotly.graph_objs as go

original = pd.read_csv("suramerica_absolute_total.csv").drop(['Unnamed: 0'], 1)
datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
datos4 = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)

paises = datos['CountryCode'].drop_duplicates().values.tolist()
pais_nombre = 'Suramerica'

preprocessing = 'imput'
search = 'original'
year_init = 2015
year_range = 20
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']
conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]

In [2]:
def estimadores_uso(df):

    df_1 = df
    temp = df['Results']
    estimadores = []

    for i in range(len(temp)):
        temp2 = temp[i] 
        r = temp2.replace(" '",",").replace("'","").replace("[[","").replace("]]","").split(']\n [')
        z = []

        for j in r:
            z.append(j.split(','))

        c = pd.DataFrame(z).groupby(1).count()[0]/len(z)

        estimadores.append([c.index.values.tolist(),c.values.tolist()])

    estimadores_df = pd.DataFrame(estimadores)
    estimadores_uso = pd.DataFrame(np.zeros((1200,4)),columns=['DTR','NONE','RFR','SVR'])

    for k in range(1200):
        for i in range(len(estimadores_df[0][k])):
            if (estimadores_df[0][k][i] == 'DTR'):
                estimadores_uso['DTR'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'NONE'):
                estimadores_uso['NONE'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'RFR'):
                estimadores_uso['RFR'][k] = estimadores_df[1][k][i]
            if (estimadores_df[0][k][i] == 'SVR'):
                estimadores_uso['SVR'][k] = estimadores_df[1][k][i]

    df_2 = estimadores_uso

    return pd.concat([df_1,df_2],axis=1)

In [3]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [4]:
def buscar_mejores(df):
    return df[(df.Resultados > 0.7)&(df.Resultados <1.0)].reset_index(drop=True)

In [5]:
def graficar_resultados(df,look_back,column,colorscale):

    trace = go.Heatmap(z=df[df['Look Back'] == look_back][column],
                       x=df[df['Look Back'] == look_back]['Base'],
                       y=df[df['Look Back'] == look_back]['Target'],
                       zmax=1.0,
                       zmin=0.0,
                       colorscale = colorscale)
    data=[trace]
    
    if(column == '%'):
        title='R2 Mean on '+pais_nombre+' with '+search+' parameters and Look Back of '+str(look_back)
    else:
        title='% of Use of ' +column+' on '+pais_nombre+' with '+search+' parameters and Look Back of '+str(look_back)
    
    layout = go.Layout(
        
        title=title,
        xaxis=dict(title='Base',   titlefont=dict(size=18)),
        yaxis=dict(title='Target', titlefont=dict(size=18))
        
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig,filename=title)

In [6]:
def buscar_resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]

    r = temp.replace(" '",",").replace("'","").replace("[[","").replace("]]","").split(']\n [')
    z = []

    for j in r:
        z.append(j.split(','))

    num = pd.DataFrame(z)[0].values.tolist()
    
    results = []
    for i in num: 
        results.append(float(i))
    
    # Códigos de los indicadores con registros 
    df_nombres = pd.DataFrame(conjunto_nombre, columns=['Resultados'])
    indicador_1 = conjunto[df_nombres[df_nombres['Resultados']==indicador_1].index.values[0]]
    indicador_2 = conjunto[df_nombres[df_nombres['Resultados']==indicador_2].index.values[0]]     
    df = pd.DataFrame(indicador_2)
    
    # Filtro indicadores repetidos
    if(indicador_1!=indicador_2):
        indicador_2 = np.array(df.loc[~df.ix[:,0].isin(indicador_1)]).flatten()
    df_zeros = tabla_base(indicador_2).dropna(thresh=5,axis=1)
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,results]),columns=['Indicadores','Resultados'])
    
    return df_results

In [7]:
def graficar_resultados_individuales(df,indicador_base,indicador_target,look_back):

    df_temp = buscar_resultados(df,indicador_base,indicador_target,look_back)

    data = [go.Bar(
                x= df_temp['Indicadores'],
                y= df_temp['Resultados'],

        marker=dict(     
            color = df_temp['Resultados'].values,
            colorscale ='Viridis',
            showscale = True
            ),
        )]

    layout = go.Layout(
        title=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back),
    )
    
    fig = go.Figure(data=data, layout=layout)

    return py.iplot(fig, filename=pais_nombre+' + '+search+': Result to predict '+indicador_target+' from '+indicador_base+' with '+'Look Back = '+str(look_back))

In [8]:
def conjuntos_predecibles(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == look_back].drop(['Time','Reg','Results','DTR','NONE','RFR','SVR'],1) , index=["Target"], columns=["Look Back"] ,aggfunc=[np.mean])
    return df_temp.sort_values([('mean','%',look_back)], ascending=False).reset_index()

In [9]:
original = estimadores_uso(original)
original

,Base,Target,Look Back,Reg,%,Time,Results,DTR,NONE,RFR,SVR
0,Agricultura,Agricultura,1,48,0.682210,31.457664,[[0.6144141076872646 'RFR' '0.68779706955']\n ...,0.295455,0.045455,0.568182,0.090909
1,Agricultura,Agricultura,2,48,0.666219,39.367144,[[0.5722749399695963 'DTR' '0.049220085144']\n...,0.227273,0.045455,0.613636,0.113636
2,Agricultura,Agricultura,3,48,0.632672,47.755963,[[0.5717784488519974 'RFR' '0.71631193161']\n ...,0.136364,0.045455,0.681818,0.136364
3,Agricultura,Ambiente,1,96,0.574538,50.047779,[[0.5462419781492407 'RFR' '0.562422990799']\n...,0.280000,0.000000,0.480000,0.240000
4,Agricultura,Ambiente,2,96,0.537096,61.782982,[[0.5131012256132508 'RFR' '0.678806781769']\n...,0.173333,0.000000,0.573333,0.253333
5,Agricultura,Ambiente,3,96,0.499742,73.107398,[[0.2455290180175272 'RFR' '0.739930868149']\n...,0.213333,0.000000,0.453333,0.333333
6,Agricultura,Ayuda,1,71,0.477034,45.658665,[[1.0 'SVR' '0.0469009876251']\n [0.7194399924...,0.196970,0.015152,0.454545,0.333333
7,Agricultura,Ayuda,2,71,0.451679,56.972980,[[1.0 'SVR' '0.0545430183411']\n [0.5595775093...,0.090909,0.015152,0.530303,0.363636
8,Agricultura,Ayuda,3,71,0.444235,68.207505,[[1.0 'SVR' '0.0565829277039']\n [0.4808817600...,0.136364,0.015152,0.378788,0.469697
9,Agricultura,Ciencia,1,13,0.703863,10.211760,[[0.8926068569204928 'DTR' '0.0458059310913']\...,0.461538,0.000000,0.384615,0.153846


In [10]:
def conjuntos_predictores(df,look_back):
    df_temp = pd.DataFrame.pivot_table( df[df['Look Back'] == look_back].drop(['Time','Reg','Results','DTR','NONE','RFR','SVR'],1) , index=["Base"], columns=["Look Back"] ,aggfunc=[np.mean])
    return df_temp.sort_values([('mean','%',look_back)], ascending=False).reset_index()

In [11]:
graficar_resultados(original,1,'DTR','Reds')

In [12]:
graficar_resultados(original,1,'RFR','Reds')

In [13]:
graficar_resultados(original,1,'SVR','Reds')

In [14]:
graficar_resultados(original,1,'NONE','Reds')

## Buscamos Mejores Resultados Individuales

In [15]:
buscar_mejores(buscar_resultados(original,'Agricultura','Infraestructura',1))

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

sortlevel is deprecated, use sort_index(level= ...)



,Indicadores,Resultados
0,"Air transport, freight (million ton-km)",0.754029
1,Container port traffic (TEU: 20 foot equivalen...,0.787286
2,Electric power consumption (kWh per capita),0.965177
3,Electric power transmission and distribution l...,0.853837
4,Electricity production from coal sources (% of...,0.84743
5,Electricity production from hydroelectric sour...,0.798194
6,Electricity production from natural gas source...,0.852833
7,Electricity production from nuclear sources (%...,0.955183
8,Electricity production from oil sources (% of ...,0.926685
9,Fixed broadband subscriptions (per 100 people),0.882516


In [16]:
buscar_mejores(buscar_resultados(original,'Agricultura','Infraestructura',1))

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

sortlevel is deprecated, use sort_index(level= ...)



,Indicadores,Resultados
0,"Air transport, freight (million ton-km)",0.754029
1,Container port traffic (TEU: 20 foot equivalen...,0.787286
2,Electric power consumption (kWh per capita),0.965177
3,Electric power transmission and distribution l...,0.853837
4,Electricity production from coal sources (% of...,0.84743
5,Electricity production from hydroelectric sour...,0.798194
6,Electricity production from natural gas source...,0.852833
7,Electricity production from nuclear sources (%...,0.955183
8,Electricity production from oil sources (% of ...,0.926685
9,Fixed broadband subscriptions (per 100 people),0.882516


## Mejores combinaciones

In [17]:
original.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results','DTR','NONE','RFR','SVR'],1)[:10]

,Base,Target,Look Back,Reg,%
0,Infraestructura,Ciencia,1,13,0.787004
1,Ciencia,Ciencia,1,13,0.784693
2,Ciencia,Ciencia,3,13,0.769557
3,Ciencia,Ciencia,2,13,0.763821
4,Comercio,Ciencia,1,11,0.760184
5,Urbano,Energia,1,49,0.755160
6,Infraestructura,Ciencia,2,13,0.749942
7,Privado,Ciencia,1,11,0.743306
8,Urbano,Urbano,1,24,0.738502
9,Energia,Urbano,1,20,0.738224


## Peores combinaciones

In [18]:
original.sort_values('%', ascending=False).reset_index().drop(['index','Time','Results','DTR','NONE','RFR','SVR'],1).tail(10)

,Base,Target,Look Back,Reg,%
1190,Pobreza,Urbano,3,21,0.140334
1191,Pobreza,Ayuda,1,69,0.140220
1192,Pobreza,Ambiente,1,111,0.135340
1193,Pobreza,Ayuda,2,69,0.134405
1194,Pobreza,Ambiente,2,111,0.132518
1195,Pobreza,Salud,2,152,0.126360
1196,Pobreza,Salud,3,152,0.124571
1197,Pobreza,Salud,1,152,0.119443
1198,Pobreza,Urbano,2,21,0.117345
1199,Pobreza,Ambiente,3,111,0.112921


## Graficamos Resultados Individuales

Resultado de predecir el conjunto de indicadores de Infraestructura desde Agricultura

In [19]:
graficar_resultados_individuales(original,'Agricultura','Infraestructura',1)

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

sortlevel is deprecated, use sort_index(level= ...)



In [20]:
graficar_resultados_individuales(original,'Infraestructura','Ciencia',1)

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated

/home/miguel/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

sortlevel is deprecated, use sort_index(level= ...)



## Graficamos Resultados Globales

Este Heatmap nos muestra el rendimiento promedio del coeficiente de determinación para cada conjunto

In [21]:
graficar_resultados(original,1,'%','Viridis')

In [22]:
graficar_resultados(original,2,'%','Viridis')

In [23]:
graficar_resultados(original,3,'%','Viridis')

# Mejores Conjuntos Base y Target

In [24]:
pd.DataFrame(np.column_stack([conjuntos_predictores(original,1)[:5],conjuntos_predictores(original,1).tail(5),conjuntos_predecibles(original,1)[:5],conjuntos_predecibles(original,1).tail(5)]), columns=[['Predictor','Predictor','Predictor','Predictor','Predictable','Predictable','Predictable','Predictable'],['Best','R2','Worse','R2','Best','R2','Worse','R2']])

Predictor                              Predictable                     \
          Best        R2    Worse        R2        Best        R2    Worse   
0       Urbano  0.613322    Deuda  0.465109     Ciencia  0.609102   Genero   
1  Agricultura  0.584875  Trabajo  0.463363      Urbano  0.603199  Trabajo   
2        Clima  0.578143    Ayuda  0.450654     Publico  0.576081    Deuda   
3     Economia  0.569819  Ciencia  0.387753     Energia  0.567925    Salud   
4      Privado  0.560036  Pobreza  0.197346    Comercio  0.566077    Ayuda   

             
         R2  
0  0.444425  
1  0.435709  
2  0.409046  
3  0.406491  
4  0.404378

In [25]:
original = pd.read_csv("suramerica_absolute_total.csv").drop(['Unnamed: 0'], 1)[:60]
grid     = pd.read_csv("suramerica_grid_total_v3.csv").drop(['Unnamed: 0'], 1)[:60]

In [26]:
def rendimiento(look_back,variable):
    original_grid   = ( (grid[grid['Look Back'] == look_back][variable].values / original[original['Look Back'] == look_back][variable].values ) - 1)*100
    return pd.DataFrame(original_grid, columns=['Original to Grid'], index=([original[original['Look Back'] == look_back]['Base'],original[original['Look Back'] == look_back]['Target']]))

In [27]:
def grafica_rendimiento(variable):

    trace0 = go.Bar(
        x=['Default Vs GridSearch'],
        y=rendimiento(1,variable).mean()[0],
        name='Look Back = 1',
        marker=dict(
            color='rgb(0,59,70)'
        )
    )
    trace1 = go.Bar(
        x=['Default Vs GridSearch'],
        y=rendimiento(2,variable).mean()[0],
        name='Look Back = 2',
        marker=dict(
            color='rgb(7,87,91)',
        )
    )

    trace2 = go.Bar(
        x=['Default Vs GridSearch'],
        y=rendimiento(3,variable).mean()[0],
        name='Look Back = 3',
        marker=dict(
            color='rgb(102,165,173)',
        )
    )

    data = [trace0, trace1, trace2]
    layout = go.Layout(
        barmode='group',
        title='Average performance improvement Time'
    )

    fig = go.Figure(data=data, layout=layout)
    
    return py.iplot(fig, filename='Performance of Time')

In [28]:
grafica_rendimiento('%')

In [29]:
grafica_rendimiento('Time')